In [8]:
import os

In [9]:
%pwd

'c:\\Users\\deepa\\Panda 2024\\Text-Summarizer-Project\\research'

In [10]:
os.chdir("../")

In [11]:
%pwd

'c:\\Users\\deepa\\Panda 2024\\Text-Summarizer-Project'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [13]:
from textSummarizer.constant import *
from textSummarizer.utils.common import read_yaml,create_directories

In [14]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self)-> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [15]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [16]:
class DataTransforamtion:
    def __init__(self,config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_example_to_features(self,example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'],max_length=128,truncation=True)

        return{
            'input_ids' : input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
            'labels': target_encoding['input_ids']}

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_example_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))



In [17]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransforamtion(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-09-19 19:16:23,815: INFO: common: yaml files: config\config.yaml loaded successfully]
[2024-09-19 19:16:23,821: INFO: common: yaml files: params.yaml loaded successfully]
[2024-09-19 19:16:23,823: INFO: common: created directory at: artifacts]
[2024-09-19 19:16:23,825: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\deepa\Panda 2024\Text-Summarizer-Project\enve\lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 104554.04 examples/s]
